### load

In [ ]:
!ls

In [ ]:
!wget http://ai.stanford.edu/~jkrause/car196/car_ims.tgz
!wget http://ai.stanford.edu/~jkrause/car196/cars_annos.mat
!tar zxf car_ims.tgz
!mkdir cars196 && mv cars_annos.mat cars196 && mv car_ims/ cars196

In [ ]:
!wget http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz
!tar zxf CUB_200_2011.tgz

In [ ]:
!pip install -q pytorch-metric-learning
!git clone -q https://github.com/tjddus9597/Proxy-Anchor-CVPR2020.git
!mv Proxy-Anchor-CVPR2020/code proxy_anchor
!sed -i 's/^import losses$/from . import losses/' proxy_anchor/utils.py

In [ ]:
from proxy_anchor.dataset import CUBirds, Cars

In [ ]:
len(CUBirds("/content", "train")), len(CUBirds("/content", "eval"))

(5864, 5924)

In [ ]:
len(Cars("/content", "train")), len(Cars("/content", "eval"))

(8054, 8131)

In [ ]:
!pip install -q umap-learn

In [ ]:
!git clone -q https://github.com/tilt/RasterFairy.git
!cd RasterFairy && pip install -q .

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import utils
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
import random
import umap
import rasterfairy
import PIL

### plot

In [ ]:
emb_full = torch.load("cars_dino_eval.pt")
len(emb_full[0])

8131

In [ ]:
num_samples = 4000
ds_x, ds_y = emb_full[0][:num_samples], emb_full[1][:num_samples]

In [ ]:
mapper = umap.UMAP(output_metric="hyperboloid", random_state=1337)
path2d = mapper.fit_transform(ds_x)
x, y = path2d[:, 0], path2d[:, 1]
z = (1 + x ** 2 + y ** 2) ** 0.5
disk_x = x / (1 + z)
disk_y = y / (1 + z)

In [ ]:
fig = plt.figure(figsize=(10, 10), clear=True)
ax = fig.add_subplot(111)
scatter = ax.scatter(disk_x, disk_y, c=ds_y, alpha=.75, s=7)

boundary = plt.Circle((0, 0), 1, fc="none", ec="k")
ax.add_patch(boundary)
ax.axis("off")
plt.show()

In [ ]:
xy = np.stack([disk_x, disk_y], axis=1)
grid_xy = rasterfairy.transformPointCloud2D(xy)

In [ ]:
grid_w, grid_h = grid_xy[1]
grid_w, grid_h

(50, 80)

In [ ]:
img_size = 50
ds_view = Cars("/content", "eval",
    transforms.Compose([transforms.Resize(img_size), transforms.CenterCrop(img_size)]))

In [ ]:
background = PIL.Image.new('RGB',(grid_w * img_size, grid_h * img_size), (255, 255, 255))
for i in range(len(grid_xy[0])):
    x, y = grid_xy[0][i]
    background.paste(ds_view[i][0], (int(x * img_size), int(y * img_size)))

In [ ]:
background.save("cars_eval.jpg", "JPEG", quality=90, optimize=True)